## Prevendo Preço da Gasolina
### Inspirado pelo @Odemir Depieri Jr
https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/serie-historica-de-precos-de-combustiveis
#### Utilizando 7 anos do histórico de dados

### Importando as Bibliotecas Necessárias

In [1]:
# Função para Trabalhar com Datas
from datetime import datetime

#Lib para Modelagem de Dados
import pandas as pd

# Lib para Análises Gráficas
import matplotlib.pyplot as plt

# Lib para acessar os recursos do Sistema Operacional
import os

# Lib para ignorar avisos
import warnings

# Aplicando regra para ignorar avisos
warnings.filterwarnings('ignore')

### Explorando uma base de dados e suas informações, antes da consolidação de todas as bases

In [2]:
# Lendo uma base de dados para entender a sua estrutura

# Local para buscar os arquivos
local = 'E:/Projetos/Python/DATASETS/Preco Gasolina/'

# Nome do arquivo
arquivo = 'ca-2020-02'

# Extensão
extensao = '.csv'

# Lendo a base de dados
base_amostra = pd.read_csv( local + arquivo + extensao, sep=';')

# Verificando as 5 primeiras linhas
base_amostra.head()

,Regiao - Sigla,Estado - Sigla,Municipio,Revenda,CNPJ da Revenda,Nome da Rua,Numero Rua,Complemento,Bairro,Cep,Produto,Data da Coleta,Valor de Venda,Valor de Compra,Unidade de Medida,Bandeira
0,SE,SP,SOROCABA,COMPETRO COMERCIO E DISTRIBUICAO DE DERIVADOS ...,00.003.188/0001-21,RUA HUMBERTO DE CAMPOS,306,NaN,JARDIM ZULMIRA,18061-000,GASOLINA,01/07/2020,"3,559","3,253",R$ / litro,BRANCA
1,SE,SP,SOROCABA,COMPETRO COMERCIO E DISTRIBUICAO DE DERIVADOS ...,00.003.188/0001-21,RUA HUMBERTO DE CAMPOS,306,NaN,JARDIM ZULMIRA,18061-000,ETANOL,01/07/2020,"2,329","2,1193",R$ / litro,BRANCA
2,SE,SP,SOROCABA,COMPETRO COMERCIO E DISTRIBUICAO DE DERIVADOS ...,00.003.188/0001-21,RUA HUMBERTO DE CAMPOS,306,NaN,JARDIM ZULMIRA,18061-000,DIESEL S10,01/07/2020,"2,859",NaN,R$ / litro,BRANCA
3,NE,BA,SALVADOR,PETROBRAS DISTRIBUIDORA S.A.,34.274.233/0015-08,RUA EDISTIO PONDE,474,NaN,STIEP,41770-395,GASOLINA,02/07/2020,"4,29","3,7676",R$ / litro,PETROBRAS DISTRIBUIDORA S.A.
4,NE,BA,SALVADOR,PETROBRAS DISTRIBUIDORA S.A.,34.274.233/0015-08,RUA EDISTIO PONDE,474,NaN,STIEP,41770-395,ETANOL,02/07/2020,"3,29","2,6651",R$ / litro,PETROBRAS DISTRIBUIDORA S.A.


In [3]:
type(base_amostra)

pandas.core.frame.DataFrame

### Filtrando uma região para analisar (SP)

In [4]:
# Gerando algumas analises

# Filtrando a região de SP
# Produto Gasolina
# Loc seleciona um subconjunto de linhas e colunas de um DataFrame

filtro_01 = base_amostra.loc[ (base_amostra['Produto'] == 'GASOLINA') &
                              (base_amostra ['Estado - Sigla'] == 'SP')]

# Verificando a Dimensão
print("Dimensão da Base de Dados:",'\n', filtro_01.shape, '\n')

# Verificando 
print(filtro_01.info())

Dimensão da Base de Dados: 
 (17952, 16) 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17952 entries, 0 to 222629
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Regiao - Sigla     17952 non-null  object
 1   Estado - Sigla     17952 non-null  object
 2   Municipio          17952 non-null  object
 3   Revenda            17952 non-null  object
 4   CNPJ da Revenda    17952 non-null  object
 5   Nome da Rua        17952 non-null  object
 6   Numero Rua         17952 non-null  object
 7   Complemento        2137 non-null   object
 8   Bairro             17907 non-null  object
 9   Cep                17952 non-null  object
 10  Produto            17952 non-null  object
 11  Data da Coleta     17952 non-null  object
 12  Valor de Venda     17952 non-null  object
 13  Valor de Compra    3089 non-null   object
 14  Unidade de Medida  17952 non-null  object
 15  Bandeira           17952 non-null  object
d

In [5]:
# Verificando campos NULOS
filtro_01.isnull().sum()

Regiao - Sigla           0
Estado - Sigla           0
Municipio                0
Revenda                  0
CNPJ da Revenda          0
Nome da Rua              0
Numero Rua               0
Complemento          15815
Bairro                  45
Cep                      0
Produto                  0
Data da Coleta           0
Valor de Venda           0
Valor de Compra      14863
Unidade de Medida        0
Bandeira                 0
dtype: int64

### Tratamento dos Dados

In [6]:
# Retirando "Virgula" e incluindo o "."
filtro_01['Valor de Venda'] = filtro_01['Valor de Venda'].str.replace(',', '.', regex=True)

# Convertendo para valor Flutuante
filtro_01['Valor de Venda'] = pd.to_numeric(filtro_01['Valor de Venda'])

# Analisando Estatísticas do Campo
filtro_01.describe()

,Valor de Venda
count,17952.000000
mean,4.050566
std,0.224615
min,3.259000
25%,3.899000
50%,3.999000
75%,4.199000
max,4.999000


In [7]:
# Convertendo a coluna para formato de Data
filtro_01['Data da Coleta'] = pd.to_datetime(filtro_01['Data da Coleta'], format="%d/%m/%Y")

# Gerando o Mês
filtro_01['Mês'] = pd.DatetimeIndex(filtro_01['Data da Coleta']).month

# Gerando o Ano
filtro_01['Ano'] = pd.DatetimeIndex(filtro_01['Data da Coleta']).year

In [8]:
# Mês e Ano Adicionados no Final
filtro_01

,Regiao - Sigla,Estado - Sigla,Municipio,Revenda,CNPJ da Revenda,Nome da Rua,Numero Rua,Complemento,Bairro,Cep,Produto,Data da Coleta,Valor de Venda,Valor de Compra,Unidade de Medida,Bandeira,Mês,Ano
0,SE,SP,SOROCABA,COMPETRO COMERCIO E DISTRIBUICAO DE DERIVADOS ...,00.003.188/0001-21,RUA HUMBERTO DE CAMPOS,306,NaN,JARDIM ZULMIRA,18061-000,GASOLINA,2020-07-01,3.559,"3,253",R$ / litro,BRANCA,7,2020
2363,SE,SP,AMPARO,AUTO POSTO DBV LTDA,09.371.227/0001-18,AVENIDA ANESIO GUIDI,344,ESQ.R.RACHID KASSOUF,DISTRITO TRES PONTES,13909-000,GASOLINA,2020-07-01,3.899,"3,4875",R$ / litro,PETROBRAS DISTRIBUIDORA S.A.,7,2020
2367,SE,SP,AMPARO,AUTO POSTO PORTAL DAS AGUAS LTDA,08.772.232/0001-70,AVENIDA WALDYR BEIRA,182,NaN,FIGUEIRA,13904-452,GASOLINA,2020-07-01,3.949,NaN,R$ / litro,PETROBRAS DISTRIBUIDORA S.A.,7,2020
2371,SE,SP,AMPARO,J M ANDRETA & CIA LTDA,48.827.125/0001-16,AVENIDA BERNARDINO DE CAMPOS,535,NaN,CENTRO,13900-400,GASOLINA,2020-07-01,3.870,"3,57",R$ / litro,IPIRANGA,7,2020
2375,SE,SP,AMPARO,J M ANDRETA & CIA LTDA,48.827.125/0002-05,RUA BENTA MARIA DE BARROS,181,NaN,ARCADAS,13908-000,GASOLINA,2020-07-01,3.870,"3,57",R$ / litro,IPIRANGA,7,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222564,SE,SP,JACAREI,AUTO POSTO JORGE MADIDI LTDA,34.179.398/0001-97,AVENIDA SIQUEIRA CAMPOS,244,NaN,CENTRO,12308-190,GASOLINA,2020-12-28,4.049,NaN,R$ / litro,BRANCA,12,2020
222579,SE,SP,FRANCA,POSTO VITORIA DE FRANCA LTDA,23.465.043/0001-48,AVENIDA BRASIL,398,NaN,VILA APARECIDA,14401-240,GASOLINA,2020-12-27,4.197,NaN,R$ / litro,PETROBRAS DISTRIBUIDORA S.A.,12,2020
222613,SE,SP,SANTO ANDRE,AUTO POSTO IMPERADOR DE SANTO ANDRE LTDA,25.356.481/0001-01,AVENIDA PRESTES MAIA,3430,TERREO,VILA GUIOMAR,09071-000,GASOLINA,2020-12-28,4.199,NaN,R$ / litro,IPIRANGA,12,2020
222622,SE,SP,AVARE,AUTO POSTO REDE LEMANS LTDA,36.710.410/0001-73,AVENIDA SALIM ANTONIO CURIATI,927,NaN,JARDIM BRASIL,18702-640,GASOLINA,2020-12-29,3.799,NaN,R$ / litro,BRANCA,12,2020


In [9]:
# Gerando a Média para o Estado de SP
analise = filtro_01.groupby(['Ano', 'Mês']).mean().reset_index()

analise

,Ano,Mês,Valor de Venda
0,2020,7,3.941447
1,2020,8,4.017643
2,2020,10,4.200923
3,2020,11,4.193360
4,2020,12,4.223971


### Consolidando TODAS as Bases de Dados, num arquivo. (Estão SEM TRATAMENTO)

In [13]:
# ------ Estrutura para Consolidar os Arquivos Baixados ------

# Local para buscar os arquivos baixados
local = 'E:/Projetos/Python/DATASETS/Preco Gasolina/'

# Variável de apoio no loop
qtd_arquivos = 0

# Loop para "Varrer" o Diretório com os arquivos baixados
for diretorio, subpastas, arquivos in os.walk(local):
    
    # loop nos arquivos
    for arquivo in arquivos:
        
        # Caso o arquivo comece com "ca-"
        if 'ca-' in str(arquivo):
            
            # Apontando para o arquivo
            arquivo_download = os.path.join(diretorio, arquivo)
            
            # Lendo a Base de Dados
            arquivo_atual = pd.read_csv(arquivo_download, sep=';')
            
            # Caso seja a primeira interação, irá criar o arquivo consolidado
            if qtd_arquivos == 0:
                base_consolidada = arquivo_atual
                
            # caso contrário, consolida
            else:
                base_consolidada = pd.concat([base_consolidada, arquivo_atual])
            
            # somando a interação
            qtd_arquivos += 1
            
print('Foi consolidado', qtd_arquivos, 'arquivos')             
       

Foi consolidado 14 arquivos


In [14]:
# Verificando a Dimensão da Base de Dados Consolidada com os 14 arquivos

base_consolidada.shape

(6755266, 16)

In [15]:
# Salvando todas as bases de dados em um arquivo csv
base_consolidada.to_csv('base_consolidada.csv')